<a href="https://colab.research.google.com/github/AnuruddhaPaul/SQUEEZE_NET_From_Scratch/blob/main/SQUEEZE_NET_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 1. Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# 2. The Fire Module
# The building block of SqueezeNet: Squeeze (1x1) -> Expand (1x1 + 3x3)
class FireModule(nn.Module):
    def __init__(self, in_channels, squeeze_planes, expand1x1_planes, expand3x3_planes):
        super(FireModule, self).__init__()

        # Squeeze Layer: 1x1 Conv to reduce depth
        self.squeeze = nn.Sequential(
            nn.Conv2d(in_channels, squeeze_planes, kernel_size=1),
            nn.ReLU(inplace=True)
        )

        # Expand Layer 1x1: 1x1 Conv to recover depth
        self.expand1x1 = nn.Sequential(
            nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1),
            nn.ReLU(inplace=True)
        )

        # Expand Layer 3x3: 3x3 Conv to capture spatial features
        self.expand3x3 = nn.Sequential(
            nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x = self.squeeze(x)
        # Concatenate the outputs of the expand layers along the channel dimension
        return torch.cat([self.expand1x1(x), self.expand3x3(x)], 1)

# 3. SqueezeNet Architecture
class SqueezeNet(nn.Module):
    def __init__(self, num_classes=10, in_channels=1):
        super(SqueezeNet, self).__init__()

        # Initial Convolution
        # Changed stride from 2 to 1 to preserve information on small MNIST images
        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            # Fire Blocks 2 & 3
            FireModule(64, 16, 64, 64),
            FireModule(128, 16, 64, 64),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            # Fire Blocks 4 & 5
            FireModule(128, 32, 128, 128),
            FireModule(256, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),

            # Fire Blocks 6, 7, 8, 9
            FireModule(256, 48, 192, 192),
            FireModule(384, 48, 192, 192),
            FireModule(384, 64, 256, 256),
            FireModule(512, 64, 256, 256),
        )

        # Final Classifier
        # SqueezeNet uses a Conv2d layer as the classifier, NOT a Linear layer.
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Conv2d(512, num_classes, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1)) # Global Average Pooling
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return torch.flatten(x, 1)

# 4. Data Preparation (MNIST)
# We resize to 96x96 to handle the 3 pooling layers comfortably
# 96 -> 47 -> 23 -> 11 -> (Global Pool) -> 1
transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 5. Initialize Model
model = SqueezeNet(num_classes=10, in_channels=1).to(device)

# 6. Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 7. Training Loop
print("Starting Training (SqueezeNet)...")
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

# 8. Evaluation
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Accuracy of SqueezeNet on MNIST: {100 * correct / total:.2f}%')

# Save Model
torch.save(model.state_dict(), 'squeezenet_mnist.pth')

Using device: cpu


100%|██████████| 9.91M/9.91M [00:00<00:00, 58.9MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.74MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.3MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 6.36MB/s]


Starting Training (SqueezeNet)...
